In [1]:
import numpy as np
# import cv2
import tifffile as tiff
from shapely.geometry import Polygon, mapping, shape, box
from skimage import io
from skimage.measure import label, regionprops
import matplotlib.pyplot as plt
# import json
from rasterio import features
import scanpy as sc
import pandas as pd
import geopandas as gpd

from matplotlib import colors

import pickle
import scipy

import seaborn as sns

from SGanalysis.SGobject import SGobject 

import matplotlib
%matplotlib inline
# matplotlib.use('Qt5Agg')
# %matplotlib qt

from sklearn.cluster import AgglomerativeClustering as AggCluster

import sys
import os

%load_ext autoreload
%autoreload 2

tools_path  = '/Users/grantkinsler/Documents/Penn/Research/SpatialBarcodes/analysis/tools/tools.py'
sys.path.append(os.path.dirname(os.path.expanduser(tools_path)))
import tools

In [2]:
# filepath  = '/Users/grantkinsler/RajLab Dropbox/Grant Kinsler/Shared_Grant/SpatialBarcodes/ImagingData/2024-02-27_spatialbarcodes_SG_expression/projects/2024-02-27_spatialbarcodes_expression/'
filepath = '/Users/grantkinsler/RajLab Dropbox/Grant Kinsler/SpatialBarcodes/ImagingData/2024-05-21_mouseexp_expression/projects/'

# roi_2
roi_name = 'roi_2'


# /roi_2

In [3]:
# locations = pd.read_csv(f'{filepath}{roi_name}/exports/cell_attributes_cell_10px_20240605.csv',index_col=0)
# matrix = pd.read_csv(f'{filepath}{roi_name}/exports/cell_by_gene_matrix_20240606_10px_cell.csv',index_col=0)

# matrix.set_index(matrix.index.astype(str),inplace=True)
# locations.set_index(matrix.index.astype(str),inplace=True)

# /Volumes/GRK_001/spatialbarcodes/2024-05-21_mouseexp_expression/projects/roi2/exports/
with open(f'{filepath}{roi_name}/exports/sg_object_20240604_dilate10.pkl', 'rb') as f:
    sg_obj = pickle.load(f)


In [4]:
# set object id (as string) as index

matrix = sg_obj.get_cell_gene_table_df()
matrix['object_id'] = [str(int(x)) for x in matrix['object_id']]
matrix.set_index('object_id',inplace=True)

sg_obj.gdf['object_id'] = [str(int(x)) for x in sg_obj.gdf['object_id']]
sg_obj.gdf.set_index('object_id',inplace=True)

No filtering available or applied.


In [5]:
sg_obj.gdf['nucleus_centroid'] = sg_obj.gdf['nucleus'].centroid.values
sg_obj.gdf['center_x'] = sg_obj.gdf['nucleus'].centroid.x.values
sg_obj.gdf['center_y'] = sg_obj.gdf['nucleus'].centroid.y.values
sg_obj.gdf['area'] = sg_obj.gdf['nucleus_dilated'].area.values

In [6]:
barcode_cols = ['bc_{:03d}'.format(i) for i in range(1,97)]

In [7]:
## Threshold barcode classification

cutoff = 3

cell_barcodes = {}
# matrix.set_index('object_id',   inplace=True)
# df.set_index('object_id')

df = matrix

# df.sort_index(inplace=True)

df['cell_id'] = df.index

for cell_id in df['cell_id']:
    this_cell = df[df['cell_id']==cell_id]
    cell_barcodes[cell_id] = []

    for bc in barcode_cols:
        if this_cell[bc].values[0] >= cutoff:
            cell_barcodes[cell_id].append(bc)

df['called_barcodes'] = cell_barcodes.values()
df['n_called_barcodes'] = [len(bc_set) for bc_set in cell_barcodes.values()]
df['barcode_names'] = ['-'.join(sorted(bc_set)) for bc_set in cell_barcodes.values()]

df = pd.merge(df,sg_obj.gdf,how='left',left_index=True,right_index=True)


In [8]:
df.to_csv(f'{filepath}{roi_name}/exports/cell_by_gene_matrix_20240606_10px_withbarcodes_atleast{cutoff}.csv')

In [20]:
with open(f'{filepath}{roi_name}/exports/sg_object_20240604_dilate10.pkl', 'rb') as f:
    sg_obj = pickle.load(f)

Exception ignored in: <function BaseGeometry.__del__ at 0x11653bdc0>
Traceback (most recent call last):
  File "/Users/grantkinsler/opt/miniconda3/envs/spatialgenomics-env2/lib/python3.9/site-packages/shapely/geometry/base.py", line 209, in __del__
    self._empty(val=None)
  File "/Users/grantkinsler/opt/miniconda3/envs/spatialgenomics-env2/lib/python3.9/site-packages/shapely/geometry/base.py", line 194, in _empty
    self._lgeos.GEOSGeom_destroy(self.__geom__)
KeyboardInterrupt: 


In [9]:
total_bc_threshold = 10

# include cells with at least 10 barcode spots
has_bcs = df[df[barcode_cols].sum(axis=1) >= 10]
matrix = has_bcs[barcode_cols]

matrix_norm = matrix.div(matrix.sum(axis=1), axis=0)
braycurtis_dist = scipy.spatial.distance.pdist(matrix_norm,metric='braycurtis')

braycurtis_dist_square = scipy.spatial.distance.squareform(braycurtis_dist)

In [10]:
threshold = 0.4
cluster = AggCluster(distance_threshold=threshold,n_clusters=None,linkage='average',affinity='precomputed').fit(braycurtis_dist_square)

In [11]:
plt.figure(figsize=(30,2))

# labels_here = [l if l.count('_') > 2 else '' for l in has_bcs['barcode_names'].values[random_indices]]
# plot_dendrogram(cluster,labels=has_bcs['barcode_names'].values[random_indices],leaf_rotation=90,leaf_font_size=6,color_threshold=0.5, truncate_mode="level",)

tools.plot_dendrogram(cluster,leaf_rotation=90,leaf_font_size=3,color_threshold=threshold, truncate_mode="level",)

# plot_dendrogram(cluster,labels=labels_here,leaf_rotation=90,leaf_font_size=2)

AttributeError: module 'tools' has no attribute 'plot_dendrogram'

<Figure size 3000x200 with 0 Axes>

In [15]:
gene_cols = [col for col in df.columns if 'bc_' not in col and col not in ['cell_id','called_barcodes','n_called_barcodes','barcode_names','area','center_x','center_y','nucleus','nucleus_centroid','nucleus_dilated']]    

adata_genes = sc.AnnData(df[gene_cols])


cols= ['cell_id','n_called_barcodes','barcode_names','called_barcodes','area','center_x','center_y']
# for col in :
#     adata_genes.obs[col] = df[col]

adata_genes.obs = df[cols]

In [16]:
cluster_name_cutoff = 3

obj_clusters = {obj_id:clus for obj_id,clus in zip(matrix.index,cluster.labels_)}

cluster_objects = {}
cluster_barcode_names = {}
cluster_found_barcodes = {}
cluster_n_found_barcodes = {}

for clu in np.unique(cluster.labels_):
    cluster_objects[clu] = matrix.index[cluster.labels_ == clu]

    avg_bc_counts = np.mean(matrix[cluster.labels_ == clu],axis=0)
    # print(avg_bc_counts)
    # break

    found_bcs = []

    for bc in barcode_cols:
        if avg_bc_counts[bc] > cutoff:
            found_bcs.append(bc)

    print(clu,found_bcs)

    cluster_found_barcodes[clu] = found_bcs
    cluster_n_found_barcodes[clu] = len(found_bcs)

    cluster_barcode_names[clu] = '-'.join(sorted(found_bcs))

0 ['bc_009', 'bc_091']
1 ['bc_085']
2 ['bc_063', 'bc_066']
3 ['bc_094']
4 ['bc_066']
5 ['bc_003', 'bc_009']
6 ['bc_067', 'bc_094']
7 ['bc_001', 'bc_029', 'bc_031', 'bc_033', 'bc_087']
8 ['bc_007', 'bc_013', 'bc_054', 'bc_060']
9 ['bc_006', 'bc_018', 'bc_043']
10 ['bc_021', 'bc_049', 'bc_079']
11 ['bc_019', 'bc_025', 'bc_074', 'bc_092']
12 ['bc_074', 'bc_083']
13 ['bc_008', 'bc_040', 'bc_067']
14 ['bc_037']
15 ['bc_001', 'bc_024', 'bc_029', 'bc_031', 'bc_066', 'bc_087', 'bc_091']
16 ['bc_016', 'bc_040', 'bc_052', 'bc_092', 'bc_093']
17 ['bc_022', 'bc_030', 'bc_060']
18 ['bc_060']
19 ['bc_001', 'bc_022', 'bc_087']
20 ['bc_026', 'bc_031', 'bc_041']
21 ['bc_021', 'bc_028', 'bc_060', 'bc_067', 'bc_074']
22 ['bc_003', 'bc_009', 'bc_066', 'bc_087']
23 ['bc_085']
24 ['bc_010', 'bc_024', 'bc_025', 'bc_041']
25 ['bc_031', 'bc_047', 'bc_093']
26 ['bc_008', 'bc_035', 'bc_059', 'bc_063', 'bc_074', 'bc_081']
27 ['bc_066', 'bc_091']
28 ['bc_008', 'bc_041', 'bc_096']
29 ['bc_016']
30 ['bc_002', 'bc_06

In [17]:
adata_genes.obs['bc_cluster'] = [obj_clusters[obj_id] if obj_id in obj_clusters else np.nan for obj_id in adata_genes.obs.index]
adata_genes.obs['bc_cluster_n_bcs'] = [cluster_n_found_barcodes[obj_clusters[obj_id]] if obj_id in obj_clusters else np.nan for obj_id in adata_genes.obs.index]
adata_genes.obs['bc_cluster_found_bcs'] = [cluster_found_barcodes[obj_clusters[obj_id]] if obj_id in obj_clusters else np.nan for obj_id in adata_genes.obs.index]
adata_genes.obs['bc_cluster_bc_names'] = [cluster_barcode_names[obj_clusters[obj_id]] if obj_id in obj_clusters else np.nan for obj_id in adata_genes.obs.index]

/var/folders/x6/q0wb9df97nj1wnk5c4t0k7840000gn/T/ipykernel_90608/3554747918.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  adata_genes.obs['bc_cluster'] = [obj_clusters[obj_id] if obj_id in obj_clusters else np.nan for obj_id in adata_genes.obs.index]
/var/folders/x6/q0wb9df97nj1wnk5c4t0k7840000gn/T/ipykernel_90608/3554747918.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  adata_genes.obs['bc_cluster_n_bcs'] = [cluster_n_found_barcodes[obj_clusters[obj_id]] if obj_id in obj_clusters else np.nan for

In [23]:
adata_genes.obs.to_csv(f'{filepath}{roi_name}/exports/cell_by_gene_matrix_20240606_10px_withbarcodes_clustering_{total_bc_threshold}bcs_{threshold}thresh.csv')


In [21]:
triple_bc_clusters = adata_genes.obs[adata_genes.obs['bc_cluster_n_bcs'] >= 3]    

cs,counts = np.unique([int(x) for x in triple_bc_clusters['bc_cluster'].values],return_counts=True)

cs[np.argsort(counts)[::-1]]

array([ 311,  372,   91, ..., 2125, 2128, 3424])